
### Dependencies and Setup


In [ ]:
knitr::opts_chunk$set(echo = TRUE)

#install.packages("ggplot2")
#install.packages("dplyr")
#install.packages(stats)
#install.packages("MASS")
#install.packages("strucchange")
#install.packages("zoo")
#install.packages("sandwich")
#install.packages("urca")
#install.packages("lmtest")
#install.packages("vars")
# 
library(ggplot2)
library(dplyr)
library(stats)
library(MASS)
library(strucchange)
library(zoo)
library(sandwich)
library(urca)
library(lmtest)
library(vars)


# Importamos Datos desde un archivo de R:

Los datos "cargados" son los originales y los ajustados por estacionalidad. Los cuales son:

* INPC: Indice Nacional de Precios al Consumidor (2QJul2018 = 100)

* TC: Tipo de Cambio FIX 

* CETE28: Tasa de rendimiento promedio mensual de los Cetes 28, en por ciento anual

* IGAE: Indicador global de la actividad económica (2013 = 100)

* IPI: Industrial Production Index (2012 = 100)


In [ ]:
load("Datos_Ad.RData")

head(Datos_Ad)


## Conversion a series de tiempo:



In [ ]:
DLINPC <- ts(log(Datos_Ad$INPC_Ad) - lag(log(Datos_Ad$INPC_Ad), k = 1), 
             start = c(2000, 1), 
             freq = 12)

DLTC <- ts(log(Datos_Ad$TC_Ad) - lag(log(Datos_Ad$TC_Ad), k = 1), 
           start = c(2000, 1), 
           freq = 12)

DLCETE28 <- ts(log(Datos_Ad$CETE28_Ad) - lag(log(Datos_Ad$CETE28_Ad), k = 1), 
               start = c(2000, 1), 
               freq = 12)

DLIGAE <- ts(log(Datos_Ad$IGAE_Ad) - lag(log(Datos_Ad$IGAE_Ad), k = 1), 
             start = c(2000, 1), 
             freq = 12)

DLIPI <- ts(log(Datos_Ad$IPI_Ad) - lag(log(Datos_Ad$IPI_Ad), k = 1), 
            start = c(2000, 1), 
            freq = 12)


# Combinacion en un solo set de datos (Dos formas)

### 1


In [ ]:
Datos <- data.frame(cbind(DLINPC, DLTC, DLCETE28, DLIGAE, DLIPI))




### 2


In [ ]:
Datos <- ts(Datos[2 : 259, ], 
            start = c(2000, 2), freq = 12)


## VAR(p) Selection:



In [ ]:
VARselect(Datos, lag.max = 12, type = "const")



## VAR Estimacion:



In [ ]:
VAR_p <- VAR(Datos, p = 2, type = "const")

summary(VAR_p)



### Por ecuación:


In [ ]:
#summary(VAR_p, equation = "DLINPC")

#summary(VAR_p, equation = "DLTC")

#summary(VAR_p, equation = "DLCETE28")

#summary(VAR_p, equation = "DLIGAE")

#summary(VAR_p, equation = "DLIPI")


### Diagnostic tests

#### Normalidad:


In [ ]:
normality.test(VAR_p)



#### Autocorrelacion Serial:

#### LAGS = 2:


In [ ]:
serial.test(VAR_p, lags.bg = 2, type = "BG")




#### LAGS = 4:


In [ ]:
serial.test(VAR_p, lags.bg = 4, type = "BG")



#### LAGS = 6:


In [ ]:
serial.test(VAR_p, lags.bg = 6, type = "BG")




#### Homocedasticidad:


In [ ]:
arch.test(VAR_p, lags.multi = 2)




### Impulse Response:


In [ ]:
? irf

#

IR_DLINPC <- irf(VAR_p, n.ahead = 12, boot = TRUE, 
                 ci = 0.95, response = "DLINPC")

IR_DLINPC


In [ ]:
plot(IR_DLINPC)



In [ ]:
IR_DLTC <- irf(VAR_p, n.ahead = 12, boot = TRUE, 
               ci = 0.95, response = "DLTC")

IR_DLTC


In [ ]:
plot(IR_DLTC)



In [ ]:
IR_DLCETE28 <- irf(VAR_p, n.ahead = 12, boot = TRUE,
                   ci = 0.95, response = "DLCETE28")

IR_DLCETE28


In [ ]:
plot(IR_DLCETE28)



In [ ]:
IR_DLIPI <- irf(VAR_p, n.ahead = 12, boot = TRUE, 
                ci = 0.95, response = "DLIPI")

IR_DLIPI


In [ ]:
plot(IR_DLIPI)



## Otras opciones del análisis impulso-respuesta

### Opción 1


In [ ]:
IR_DLINPC_2 <- irf(VAR_p, n.ahead = 12, boot = TRUE, 
                   ci = 0.95, response = "DLINPC",
                   ortho = TRUE, cumulative = FALSE)

plot(IR_DLINPC_2)



### Opción 2


In [ ]:
IR_DLINPC_3 <- irf(VAR_p, n.ahead = 12, boot = TRUE, 
                   ci = 0.95, response = "DLINPC",
                   ortho = TRUE, cumulative = TRUE)

plot(IR_DLINPC_3)
